In [1]:
import pandas as pd
import ast
import numpy as np

from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()

import os
import openai

In [2]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

False

In [3]:
from preprocessing import *
from prompt_functions import create_pt_prompt_per_day, create_pt_prompt_per_service

In [6]:
labs = pd.read_csv('C:/Users/tthua/Box/DischargeMe_BioNLP2024/hosp/labevents.csv.gz', nrows = 1000)

In [7]:
labs.head(100)

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,1,10000032,NaN,45421181,51237,P28Z0X,2180-03-23 11:51:00,2180-03-23 15:15:00,1.4,1.4,NaN,0.9,1.1,abnormal,ROUTINE,NaN
1,2,10000032,NaN,45421181,51274,P28Z0X,2180-03-23 11:51:00,2180-03-23 15:15:00,___,15.1,sec,9.4,12.5,abnormal,ROUTINE,VERIFIED.
2,3,10000032,NaN,52958335,50853,P28Z0X,2180-03-23 11:51:00,2180-03-25 11:06:00,___,15.0,ng/mL,30.0,60.0,abnormal,ROUTINE,NEW ASSAY IN USE ___: DETECTS D2 AND D3 25-OH ...
3,4,10000032,NaN,52958335,50861,P28Z0X,2180-03-23 11:51:00,2180-03-23 16:40:00,102,102.0,IU/L,0.0,40.0,abnormal,ROUTINE,NaN
4,5,10000032,NaN,52958335,50862,P28Z0X,2180-03-23 11:51:00,2180-03-23 16:40:00,3.3,3.3,g/dL,3.5,5.2,abnormal,ROUTINE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,10000032,NaN,43430170,50863,NaN,2180-05-06 22:25:00,2180-05-06 23:16:00,114,114.0,IU/L,35.0,105.0,abnormal,STAT,NaN
96,97,10000032,NaN,43430170,50868,NaN,2180-05-06 22:25:00,2180-05-06 23:16:00,9,9.0,mEq/L,8.0,20.0,NaN,STAT,NaN
97,98,10000032,NaN,43430170,50878,NaN,2180-05-06 22:25:00,2180-05-06 23:16:00,114,114.0,IU/L,0.0,40.0,abnormal,STAT,NaN
98,99,10000032,NaN,43430170,50882,NaN,2180-05-06 22:25:00,2180-05-06 23:16:00,27,27.0,mEq/L,22.0,32.0,NaN,STAT,NaN


In [5]:
labs_icd = pd.read_csv('C:/Users/tthua/Box/DischargeMe_BioNLP2024/hosp/d_labitems.csv.gz')

In [14]:
labs = labs.merge(labs_icd, on="itemid", how="left")

In [15]:
labs

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments,label,fluid,category
0,1,10000032,NaN,45421181,51237,P28Z0X,2180-03-23 11:51:00,2180-03-23 15:15:00,1.4,1.4,NaN,0.9,1.1,abnormal,ROUTINE,NaN,INR(PT),Blood,Hematology
1,2,10000032,NaN,45421181,51274,P28Z0X,2180-03-23 11:51:00,2180-03-23 15:15:00,___,15.1,sec,9.4,12.5,abnormal,ROUTINE,VERIFIED.,PT,Blood,Hematology
2,3,10000032,NaN,52958335,50853,P28Z0X,2180-03-23 11:51:00,2180-03-25 11:06:00,___,15.0,ng/mL,30.0,60.0,abnormal,ROUTINE,NEW ASSAY IN USE ___: DETECTS D2 AND D3 25-OH ...,25-OH Vitamin D,Blood,Chemistry
3,4,10000032,NaN,52958335,50861,P28Z0X,2180-03-23 11:51:00,2180-03-23 16:40:00,102,102.0,IU/L,0.0,40.0,abnormal,ROUTINE,NaN,Alanine Aminotransferase (ALT),Blood,Chemistry
4,5,10000032,NaN,52958335,50862,P28Z0X,2180-03-23 11:51:00,2180-03-23 16:40:00,3.3,3.3,g/dL,3.5,5.2,abnormal,ROUTINE,NaN,Albumin,Blood,Chemistry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,998,10000117,NaN,45886874,51519,P73N8R,2174-06-03 09:00:00,2174-06-03 19:21:00,NONE,NaN,NaN,NaN,NaN,NaN,ROUTINE,NaN,Yeast,Urine,Hematology
996,999,10000117,NaN,47027790,50931,P73N8R,2174-06-03 09:00:00,2174-06-03 19:02:00,78,78.0,mg/dL,70.0,105.0,NaN,ROUTINE,NaN,Glucose,Blood,Chemistry
997,1000,10000117,NaN,79301516,50861,P73N8R,2174-06-03 09:00:00,2174-06-03 19:57:00,18,18.0,IU/L,0.0,40.0,NaN,ROUTINE,NaN,Alanine Aminotransferase (ALT),Blood,Chemistry
998,1001,10000117,NaN,79301516,50862,P73N8R,2174-06-03 09:00:00,2174-06-03 19:57:00,4.7,4.7,g/dL,3.4,4.8,NaN,ROUTINE,NaN,Albumin,Blood,Chemistry


Subsetting the labs

In [ ]:
edstay = pd.read_csv('raw/train/edstays.csv')

In [ ]:
cohort_subject_ids = edstay['subject_id'].unique()
cohort_hadm_ids = edstay['hadm_id'].unique()
cohort_stay_ids = edstay['stay_id'].unique()

In [16]:
labs = labs.astype({"charttime":"datetime64[ns]",
                      "storetime":"datetime64[ns]"})

In [24]:
# Filtering labs for only abnormal lab values
filtered_labs = labs[(labs['valuenum'] < labs['ref_range_lower']) | (labs['valuenum'] > labs['ref_range_upper'])]


In [25]:
# TODO THOMAS: figure out how to better encapsulate lab values
filtered_labs['text'] = (filtered_labs['category'].fillna("") + " " + filtered_labs['fluid'].fillna("") + " " 
                + filtered_labs['label'].fillna("") + ": " + filtered_labs['value'].fillna("") + " " 
                + filtered_labs['valueuom'].fillna(""))

C:\Users\tthua\AppData\Local\Temp\ipykernel_15972\1574885110.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_labs['text'] = (filtered_labs['category'].fillna("") + " " + filtered_labs['fluid'].fillna("") + " "


In [26]:
filtered_labs

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments,label,fluid,category,text
0,1,10000032,NaN,45421181,51237,P28Z0X,2180-03-23 11:51:00,2180-03-23 15:15:00,1.4,1.400,NaN,0.900,1.100,abnormal,ROUTINE,NaN,INR(PT),Blood,Hematology,Hematology Blood INR(PT): 1.4
1,2,10000032,NaN,45421181,51274,P28Z0X,2180-03-23 11:51:00,2180-03-23 15:15:00,___,15.100,sec,9.400,12.500,abnormal,ROUTINE,VERIFIED.,PT,Blood,Hematology,Hematology Blood PT: ___ sec
2,3,10000032,NaN,52958335,50853,P28Z0X,2180-03-23 11:51:00,2180-03-25 11:06:00,___,15.000,ng/mL,30.000,60.000,abnormal,ROUTINE,NEW ASSAY IN USE ___: DETECTS D2 AND D3 25-OH ...,25-OH Vitamin D,Blood,Chemistry,Chemistry Blood 25-OH Vitamin D: ___ ng/mL
3,4,10000032,NaN,52958335,50861,P28Z0X,2180-03-23 11:51:00,2180-03-23 16:40:00,102,102.000,IU/L,0.000,40.000,abnormal,ROUTINE,NaN,Alanine Aminotransferase (ALT),Blood,Chemistry,Chemistry Blood Alanine Aminotransferase (ALT)...
4,5,10000032,NaN,52958335,50862,P28Z0X,2180-03-23 11:51:00,2180-03-23 16:40:00,3.3,3.300,g/dL,3.500,5.200,abnormal,ROUTINE,NaN,Albumin,Blood,Chemistry,Chemistry Blood Albumin: 3.3 g/dL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,938,10000084,NaN,91021189,51498,NaN,2160-12-28 00:00:00,2160-12-28 01:21:00,1.037,1.037,,1.001,1.035,abnormal,STAT,NaN,Specific Gravity,Urine,Hematology,Hematology Urine Specific Gravity: 1.037
939,942,10000084,NaN,91021189,51514,NaN,2160-12-28 00:00:00,2160-12-28 01:21:00,2,2.000,mg/dL,0.200,1.000,abnormal,STAT,NaN,Urobilinogen,Urine,Hematology,Hematology Urine Urobilinogen: 2 mg/dL
954,957,10000108,NaN,93772874,51256,NaN,2163-09-27 18:55:00,2163-09-27 19:12:00,72.9,72.900,%,50.000,70.000,abnormal,STAT,NaN,Neutrophils,Blood,Hematology,Hematology Blood Neutrophils: 72.9 %
977,980,10000117,NaN,32208337,50965,P73N8R,2174-06-03 09:00:00,2174-06-03 19:43:00,111,111.000,pg/mL,15.000,65.000,abnormal,ROUTINE,NaN,Parathyroid Hormone,Blood,Chemistry,Chemistry Blood Parathyroid Hormone: 111 pg/mL


In [30]:
filtered_labs["comments"].value_counts()

comments
IF FASTING, 70-100 NORMAL, >125 PROVISIONAL DIABETES.        10
___                                                          10
VERIFIED BY SMEAR.                                            2
VERIFIED BY ALTERNATE METHODOLOGY.                            2
VERIFIED.                                                     1
NEW ASSAY IN USE ___: DETECTS D2 AND D3 25-OH ACCURATELY.     1
MEASURED BY ___.                                              1
HEMOLYSIS FALSELY ELEVATES K..                                1
Hemolysis falsely decreases this test.                        1
Hemolysis falsely elevates this test.                         1
If fasting, 70-100 normal, >125 provisional diabetes.         1
Name: count, dtype: int64

In [33]:
labs[labs["flag"] == "abnormal"]

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments,label,fluid,category,text
0,1,10000032,NaN,45421181,51237,P28Z0X,2180-03-23 11:51:00,2180-03-23 15:15:00,1.4,1.4,NaN,0.9,1.1,abnormal,ROUTINE,NaN,INR(PT),Blood,Hematology,Hematology Blood INR(PT): 1.4
1,2,10000032,NaN,45421181,51274,P28Z0X,2180-03-23 11:51:00,2180-03-23 15:15:00,___,15.1,sec,9.4,12.5,abnormal,ROUTINE,VERIFIED.,PT,Blood,Hematology,Hematology Blood PT: ___ sec
2,3,10000032,NaN,52958335,50853,P28Z0X,2180-03-23 11:51:00,2180-03-25 11:06:00,___,15.0,ng/mL,30.0,60.0,abnormal,ROUTINE,NEW ASSAY IN USE ___: DETECTS D2 AND D3 25-OH ...,25-OH Vitamin D,Blood,Chemistry,Chemistry Blood 25-OH Vitamin D: ___ ng/mL
3,4,10000032,NaN,52958335,50861,P28Z0X,2180-03-23 11:51:00,2180-03-23 16:40:00,102,102.0,IU/L,0.0,40.0,abnormal,ROUTINE,NaN,Alanine Aminotransferase (ALT),Blood,Chemistry,Chemistry Blood Alanine Aminotransferase (ALT)...
4,5,10000032,NaN,52958335,50862,P28Z0X,2180-03-23 11:51:00,2180-03-23 16:40:00,3.3,3.3,g/dL,3.5,5.2,abnormal,ROUTINE,NaN,Albumin,Blood,Chemistry,Chemistry Blood Albumin: 3.3 g/dL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,941,10000084,NaN,91021189,51512,NaN,2160-12-28 00:00:00,2160-12-28 01:21:00,NaN,NaN,/hpf,NaN,NaN,abnormal,STAT,RARE*.,Urine Mucous,Urine,Hematology,Hematology Urine Urine Mucous: /hpf
939,942,10000084,NaN,91021189,51514,NaN,2160-12-28 00:00:00,2160-12-28 01:21:00,2,2.0,mg/dL,0.2,1.0,abnormal,STAT,NaN,Urobilinogen,Urine,Hematology,Hematology Urine Urobilinogen: 2 mg/dL
954,957,10000108,NaN,93772874,51256,NaN,2163-09-27 18:55:00,2163-09-27 19:12:00,72.9,72.9,%,50.0,70.0,abnormal,STAT,NaN,Neutrophils,Blood,Hematology,Hematology Blood Neutrophils: 72.9 %
977,980,10000117,NaN,32208337,50965,P73N8R,2174-06-03 09:00:00,2174-06-03 19:43:00,111,111.0,pg/mL,15.0,65.0,abnormal,ROUTINE,NaN,Parathyroid Hormone,Blood,Chemistry,Chemistry Blood Parathyroid Hormone: 111 pg/mL
